<div class="usecase-title">Self Designed Walking Tour</div>

<div class="usecase-authors"><b>Authored by: </b> Amy Tran</div>

<div class="usecase-duration"><b>Duration:</b> 60 mins</div>

<div class="usecase-level-skill">
    <div class="usecase-level"><b>Level: </b>Intermediate</div>
    <div class="usecase-skill"><b>Pre-requisite Skills: </b>Python</div>
</div>

<div class="usecase-section-header">Scenario</div>

As international and domestic tourists are returning to Melbourne after Covid, a tool that helps visitors create their own walking experience is beneficial. Based on the visitor’s preference, the tool will suggest appropriate walking route that incorporates landmarks or places of interest. 

<div class="usecase-section-header">What this use case will teach you</div>

At the end of this use case you will:
- Load relevant datasets required for analysis
- Map city landmark

<div class="usecase-section-header">Python Libraries</div>

In [359]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
import plotly.express as px
import geopy.distance
import folium
from folium import plugins
from PIL import Image, ImageDraw

#pip install plotly==5.8.0
#pip install geopy
#pip install folium

<div class="usecase-section-header">Data Extraction</div>

In [360]:
def get_data(base, url, size = 0):
    target_filters = f'records?limit={10}&offset={size}&timezone=UTC'
    target_url = f'{base}{url}/{target_filters}'
    result = session.get(target_url+f'&apikey={API_KEY}')
    status_code = result.status_code
    if status_code == 200:
        result_json = result.json()
        max_results = result_json['total_count']
        links = result_json['links']
        records = result_json['records']
        records_df = pd.json_normalize(records)
    
        #Update column labels
        records_df.drop(columns=['links'],inplace=True)
        column_names = records_df.columns.values.tolist()

        #Replace geolocation.lat & geolocation.lon
        column_names = ['_'.join((a.split(".")[-2:])) if a.split('.')[-2]=='geolocation' else a for a in column_names]
        column_names = [i.split('.')[-1] for i in column_names]
        records_df.columns = column_names
    
        next_url = None
             
        #Obtain next url
        if records_df.shape[0] != max_results:
            for l in links:
                if l['rel'] == 'next':
                    next_url = l['href']
    
        return[records_df, next_url, column_names, max_results, status_code]
    else: return[None, None, None, None, status_code]

In [361]:
#Extract self guided walks data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'self-guided-walks'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
walks = data.copy()
#Rename Longitude and Latitude columns
walks = walks.rename(columns = {'geo_point_2d.lon' : 'lon', 'geo_point_2d.lat' : 'lat'})
print('Download completed:', walks.shape[0], 'rows,', walks.shape[1], 'columns extracted' )

Download completed: 8 rows, 9 columns extracted


In [362]:
#Extract landmarks and places of interest data 
session = requests.Session()
base = 'https://data.melbourne.vic.gov.au/api/v2/catalog/datasets/'
url = 'landmarks-and-places-of-interest-including-schools-theatres-health-services-spor'

target_url = f'{base}{url}/exports/json'
result = session.get(target_url)
result_json = result.json()
data = pd.json_normalize(result_json)
landmarks = data.copy()
#Rename Longitude and Latitude columns
landmarks = landmarks.rename(columns = {'co_ordinates.lon' : 'lon', 'co_ordinates.lat' : 'lat'})
print('Download completed:', landmarks.shape[0], 'rows,', landmarks.shape[1], 'columns extracted' )

Download completed: 242 rows, 5 columns extracted


In [364]:
#Select a walk
my_walk = 'Melbourne Music Walk'
selected_walk = walks[(walks['name'] == my_walk)]
selected_walk = selected_walk.loc[:,'geo_shape.geometry.coordinates']

no_of_coordinates = len(selected_walk[0][0])
coordinate_lon = []
coordinate_lat = []

no_of_coordinates = len(selected_walk[0][0])

for i in range(no_of_coordinates):
    lon = selected_walk[0][0][i][0]
    lat = selected_walk[0][0][i][1]
    coordinate_lon.append(lon)
    coordinate_lat.append(lat)

#Creat a dataframe containing each pair of latitude and longitude  in walk coordinates
walk_coordinates = pd.DataFrame({'lon': coordinate_lon, 'lat': coordinate_lat})

In [365]:
#Select landmark categories of interest
my_landmarks = ['Theatre Live','Art Gallery/Museum']
selected_landmarks = landmarks[landmarks.sub_theme.isin(my_landmarks)]
selected_landmarks.reset_index(drop = True, inplace = True)

In [366]:
#Create function to calculate distance between 2 coordinates
import geopy.distance
def distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    km = geopy.distance.geodesic(coords_1, coords_2).km
    return (km)

In [367]:
#Calculate distance (km) between each walk coordinates to the landmarks of interest
distance_parameter = 0.5 #Set distance parameter

#len(walk_coordinates)
#len(selected_landmarks)

all_suggestions = pd.DataFrame()

for i in range(len(walk_coordinates)):
    suggested_landmarks = []
    suggested_landmarks_lon = []
    suggested_landmarks_lat = []
    suggested_landmarks_walk_distance = []
    
    for j in range(len(selected_landmarks)):
        walk_distance = distance(walk_coordinates.lat[i], walk_coordinates.lon[i], selected_landmarks.lat[j], selected_landmarks.lon[j])
        if walk_distance <= distance_parameter:
            suggested_landmarks.append(selected_landmarks.feature_name[j])
            suggested_landmarks_lon.append(selected_landmarks.lon[j])
            suggested_landmarks_lat.append(selected_landmarks.lat[j])
            suggested_landmarks_walk_distance.append(walk_distance)

    suggestions = pd.DataFrame({'landmarks': suggested_landmarks, 'lon': suggested_landmarks_lon,
                                'lat': suggested_landmarks_lat, 'walking_distance':suggested_landmarks_walk_distance})
    
    #Select top 3 suggested landmarks based on shortest walking distance
    top3_suggestions = suggestions.sort_values('walking_distance', ascending = True).head(3)

    all_suggestions = pd.concat([all_suggestions, top3_suggestions])
    all_suggestions.reset_index(drop = True, inplace = True)
    
final_suggestions = all_suggestions[['landmarks', 'lon','lat']]
final_suggestions.drop_duplicates(inplace = True)
final_suggestions.reset_index(drop = True, inplace = True)


<ipython-input-367-4bbf2b55c145>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_suggestions.drop_duplicates(inplace = True)


In [428]:
#Create map visualisation
map = folium.Map(location = [final_suggestions.lat.mean(), final_suggestions.lon.mean()], zoom_start=15, control_scale=True)

#Add the walking path
walk_coordinates_lats_longs = walk_coordinates[['lat','lon']].values
plugins.AntPath(walk_coordinates_lats_longs).add_to(map)

#Add landmarks and place of interest
for index, location_info in final_suggestions.iterrows():
    folium.Marker([location_info['lat'], location_info['lon']], popup=location_info['landmarks']).add_to(map)

#Display map
display(map)